In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
# Logging
# from __main__ import logger_name
logger_name = 'UTIL LOGGER'
import logging
log = logging.getLogger(logger_name)

In [27]:
zone_time_map = {
  'asia-east1': 'Asia/Taipei',
  'asia-east2': 'Asia/Hong_Kong',
  'asia-northeast1': 'Asia/Tokyo',
  'asia-northeast2': 'Asia/Tokyo',
  'asia-northeast3': 'Asia/Seoul',
  'asia-south1': 'Asia/Kolkata',
  'asia-southeast1': 'Asia/Singapore',
  'asia-southeast2': 'Asia/Jakarta',
  'australia-southeast1': 'Australia/Sydney',
  'europe-north1': 'Europe/Helsinki',
  'europe-west1': 'Europe/Brussels',
  'europe-west2': 'Europe/London',
  'europe-west3': 'Europe/Berlin',
  'europe-west4': 'Europe/Amsterdam',
  'europe-west6': 'Europe/Zurich',
  'northamerica-northeast1': 'America/Montreal',
  'southamerica-east1': 'America/Sao_Paulo',
  'us-central1': 'US/Central',
  'us-east1': 'US/Eastern',
  'us-east4': 'US/Eastern',
  'us-west1': 'US/Pacific',
  'us-west2': 'US/Pacific',
  'us-west3': 'US/Mountain',
  'us-west4': 'US/Pacific'
}


In [28]:
def min_percent_index(num, percent, offset):
    min_index = round((num-1)*percent) + offset
    if min_index >= num:
        min_index = num-1
    return min_index

def max_percent_index(num, percent, offset):
    max_index = round((num-1)*percent) + offset
    if max_index >= num:
        max_index = num-1
    return max_index

def kernel_version(kernel_release):
    kernel_split = kernel_release.split('.')
    version = float(kernel_split[0] + '.' + kernel_split[1])
    return version

In [29]:
filename = 'data/bq_results_09082022.csv'
trainpercent = 0.6
validpercent = 0.2
horizon=0
window=10
normalize=0

# query='vm_1_gce_network_tier == "premium"'
query = 'vm_1_gce_network_tier == "premium" and vm_1_machine_type == "n1-standard-16" and ip_type == "internal" and tcp_congestion_control == "bbr"'

In [30]:
df = pd.read_csv(filename, dtype={"ping_average_latency": float, 
                                      "iperf_throughput_1_thread": float,
                                      "iperf_throughput_32_threads": float})

################# Data prep #################
df[['vm_1_gce_network_tier',
    'vm_2_gce_network_tier']] = df[['vm_1_gce_network_tier',
                                    'vm_2_gce_network_tier']].fillna(value='premium')
df[['tcp_max_receive_buffer']] = df[['tcp_max_receive_buffer']].fillna(value=6291456)
# df = df.set_index('thedate')



In [31]:
# Filter the data to what we want

df = df[df['iperf_throughput_1_thread'].notnull()]
df = df[df['ping_average_latency'].notnull()]
df = df[df['sending_zone'].notnull()]
df = df[df['receiving_zone'].notnull()]
# df = df[df['vm_1_cloud'].notnull()]
# df = df[df['vm_2_cloud'].notnull()]
df[['pandas_datetime']] = df[['thedate']].apply(pd.to_datetime)

# Reduce dataset to just recent where it is more stable
df = df[df['pandas_datetime'] > '03/03/2022']
df.reset_index(inplace=True,drop=True)

In [32]:
# Get local time
df['sending_zone_datetime'] = df.apply(lambda row: row['pandas_datetime'].tz_convert(zone_time_map[row['sending_zone'][:-2]]), axis=1)
df['receiving_zone_datetime'] = df.apply(lambda row: row['pandas_datetime'].tz_convert(zone_time_map[row['receiving_zone'][:-2]]), axis=1)

# Remove timezone data
df['sending_zone_datetime'] = df.apply(lambda row: pd.to_datetime(row['sending_zone_datetime'].tz_localize(None).strftime("%Y/%m/%d, %H:%M:%S")), axis=1)
df['receiving_zone_datetime'] = df.apply(lambda row: pd.to_datetime(row['receiving_zone_datetime'].tz_localize(None).strftime("%Y/%m/%d, %H:%M:%S")), axis=1)

# extract hour of the day and day of the week
df['sending_zone_day'] = df['sending_zone_datetime'].dt.dayofweek
df['receiving_zone_day'] = df['receiving_zone_datetime'].dt.dayofweek
df['sending_zone_hour'] = df['sending_zone_datetime'].dt.hour
df['receiving_zone_hour'] = df['receiving_zone_datetime'].dt.hour

# convert hour of the day and day of the week into sin and cos components because they are cyclical
df['sending_zone_day_cos'] = np.cos(2*np.pi*df['sending_zone_day']/7)
df['sending_zone_day_sin'] = np.sin(2*np.pi*df['sending_zone_day']/7)

df['receiving_zone_day_cos'] = np.cos(2*np.pi*df['receiving_zone_day']/7)
df['receiving_zone_day_sin'] = np.sin(2*np.pi*df['receiving_zone_day']/7)

df['sending_zone_hour_cos'] = np.cos(2*np.pi*df['sending_zone_hour']/24)
df['sending_zone_hour_sin'] = np.sin(2*np.pi*df['sending_zone_hour']/24)

df['receiving_zone_hour_cos'] = np.cos(2*np.pi*df['receiving_zone_hour']/24)
df['receiving_zone_hour_sin'] = np.sin(2*np.pi*df['receiving_zone_hour']/24)

In [33]:
df['kernel_version'] = df['vm_1_kernel_release'].apply(kernel_version)
df.sort_values(by=['iperf_timestamp', 'run_uri'], inplace=True, ascending=True)
df = df.set_index('thedate')
one_hot_machine_type = pd.get_dummies(df['vm_1_machine_type'], dtype='float32')
df = df.join(one_hot_machine_type)
one_hot_ip_type = pd.get_dummies(df['ip_type'], dtype='float32')
df = df.join(one_hot_ip_type)

df['vm_1_os_info_trunc'] = df['vm_1_os_info'].str.extract(r'(\w+\s+\d+.\d+).\d+\s+\w+')

one_hot_congestion_control = pd.get_dummies(df['tcp_congestion_control'], dtype='float32')
df = df.join(one_hot_congestion_control)

In [34]:
df['group_mean'] = df.groupby(['sending_zone',
                   'receiving_zone',
                   'tcp_max_receive_buffer',
                   'vm_1_machine_type',
                   'ip_type',
                   'tcp_congestion_control',
                   'vm_1_os_info_trunc'
                    ])['iperf_throughput_32_threads'].transform('mean')

df['group_stddev'] = df.groupby(['sending_zone',
                       'receiving_zone',
                       'tcp_max_receive_buffer',
                       'vm_1_machine_type',
                       'ip_type',
                       'tcp_congestion_control',
                       'vm_1_os_info_trunc'
                        ])['iperf_throughput_32_threads'].transform('std')


df['group_id'] = df.groupby(['sending_zone',
                       'receiving_zone',
                       'tcp_max_receive_buffer',
                       'vm_1_machine_type',
                       'ip_type',
                       'tcp_congestion_control',
                       'vm_1_os_info_trunc'
                        ])['iperf_throughput_32_threads'].ngroup()


In [35]:
df = df.query(query)

In [36]:
df.to_csv('data/bq_results_reduced.csv')


In [37]:
# TODO rethink how are are normalizing data
# maybe we want to do it per group?
# normalize data
mean = df['ping_average_latency'].mean(axis=0)
df[['ping_average_latency']] = df[['ping_average_latency']] - mean
std = df['ping_average_latency'].std(axis=0)
df[['ping_average_latency']] /= std

mean = df['iperf_throughput_1_thread'].mean(axis=0)
df[['iperf_throughput_1_thread']] = df[['iperf_throughput_1_thread']] - mean
std = df['iperf_throughput_1_thread'].std(axis=0)
df[['iperf_throughput_1_thread']] /= std

mean = df['iperf_throughput_32_threads'].mean(axis=0)
iperf_32_thread_mean = mean
df[['iperf_throughput_32_threads']] = df[['iperf_throughput_32_threads']] - mean
std = df['iperf_throughput_32_threads'].std(axis=0)
iperf_32_thread_std = std
df[['iperf_throughput_32_threads']] /= std

mean = df['tcp_max_receive_buffer'].mean(axis=0)
mean = mean + 0.000000001
df[['tcp_max_receive_buffer']] = df[['tcp_max_receive_buffer']] - mean
std = df['tcp_max_receive_buffer'].std(axis=0)
std = std + 0.000000001
df[['tcp_max_receive_buffer']] /= std

mean = df['kernel_version'].mean(axis=0)
df[['kernel_version']] = df[['kernel_version']] - mean
std = df['kernel_version'].std(axis=0)
df[['kernel_version']] /= std

In [38]:
columns   = ['pandas_datetime',
            'iperf_throughput_1_thread',
            'iperf_throughput_32_threads',
            'ping_average_latency',
            'tcp_max_receive_buffer',
            'sending_zone_day_cos',
            'sending_zone_day_sin',
            'receiving_zone_day_cos',
            'receiving_zone_day_sin',
            'sending_zone_hour_cos',
            'sending_zone_hour_sin',
            'receiving_zone_hour_cos',
            'receiving_zone_hour_sin', 
            'kernel_version',
            'group_mean',
            'group_stddev'
            # 'bbr', 'cubic',
            # 'n1-standard-16', 'n1-standard-2',
            # 'external', 'internal',
            # 'Ubuntu 14.04', 'Ubuntu 16.04', 'Ubuntu 18.04', 'Ubuntu 20.04',
            # 'vm_1_machine_type',
            # 'ip_type',
            # 'tcp_congestion_control',
            # 'vm_1_os_info_trunc'
           ]

In [39]:
gb = df.groupby(['sending_zone',
                 'receiving_zone',
                 'tcp_max_receive_buffer',
                 'vm_1_machine_type',
                 'ip_type',
                 'tcp_congestion_control',
                 'vm_1_os_info_trunc'], 
                    as_index=False)[columns]
    
groups = list(gb.groups)

rawdata = gb

log.debug("End reading data")

w         = window
h         = horizon
m         = rawdata.get_group(groups[0]).shape[-1]
data      = None
n         = None
normalize = normalize
scale     = np.ones(m)

In [42]:
len(gb.groups)

552

In [48]:
df['vm_1_os_info_trunc'].unique()

array(['Ubuntu 20.04'], dtype=object)

In [40]:
gb.groups

{('asia-east1-b', 'asia-east2-a', 0.0, 'n1-standard-16', 'internal', 'bbr', 'Ubuntu 20.04'): ['2022-03-03 04:11:17.582279 UTC', '2022-03-04 06:55:46.81243 UTC', '2022-03-05 10:11:20.458815 UTC', '2022-03-06 13:16:17.535309 UTC', '2022-03-07 15:57:23.335103 UTC', '2022-03-08 19:07:16.288104 UTC', '2022-03-09 22:22:28.062353 UTC', '2022-03-11 01:06:07.065495 UTC', '2022-03-12 04:17:12.899867 UTC', '2022-03-13 07:17:41.524483 UTC', '2022-03-14 10:15:48.29161 UTC', '2022-03-15 13:15:39.680529 UTC', '2022-03-16 16:16:57.966655 UTC', '2022-03-17 19:16:36.607068 UTC', '2022-03-18 22:06:44.291148 UTC', '2022-03-20 01:09:24.810893 UTC', '2022-03-21 04:11:08.767901 UTC', '2022-03-22 08:02:42.589793 UTC', '2022-03-23 10:11:09.131451 UTC', '2022-03-24 13:16:56.986729 UTC', '2022-03-25 16:27:02.754188 UTC', '2022-03-26 19:26:42.655122 UTC', '2022-03-27 22:01:34.621864 UTC', '2022-03-29 02:59:42.325118 UTC', '2022-03-30 04:26:45.009135 UTC', '2022-03-31 07:22:34.867579 UTC', '2022-04-01 19:27:56.066

In [41]:
rawdata.get_group(groups[0])

,pandas_datetime,iperf_throughput_1_thread,iperf_throughput_32_threads,ping_average_latency,tcp_max_receive_buffer,sending_zone_day_cos,sending_zone_day_sin,receiving_zone_day_cos,receiving_zone_day_sin,sending_zone_hour_cos,sending_zone_hour_sin,receiving_zone_hour_cos,receiving_zone_hour_sin,kernel_version,group_mean,group_stddev
thedate,,,,,,,,,,,,,,,,
2022-03-03 04:11:17.582279 UTC,2022-03-03 04:11:17.582279+00:00,0.390799,-0.726131,-1.661377,0.0,-0.900969,0.433884,-0.900969,0.433884,-1.000000e+00,1.224647e-16,-1.000000e+00,1.224647e-16,-2.675964,25923.007194,2176.550257
2022-03-04 06:55:46.81243 UTC,2022-03-04 06:55:46.812430+00:00,0.659340,-0.311003,-1.661595,0.0,-0.900969,-0.433884,-0.900969,-0.433884,-8.660254e-01,-5.000000e-01,-8.660254e-01,-5.000000e-01,-2.675964,25923.007194,2176.550257
2022-03-05 10:11:20.458815 UTC,2022-03-05 10:11:20.458815+00:00,1.474641,0.416758,-1.656420,0.0,-0.222521,-0.974928,-0.222521,-0.974928,-1.836970e-16,-1.000000e+00,-1.836970e-16,-1.000000e+00,-0.420851,25923.007194,2176.550257
2022-03-06 13:16:17.535309 UTC,2022-03-06 13:16:17.535309+00:00,1.043686,-0.331600,-1.661634,0.0,0.623490,-0.781831,0.623490,-0.781831,7.071068e-01,-7.071068e-01,7.071068e-01,-7.071068e-01,-0.420851,25923.007194,2176.550257
2022-03-07 15:57:23.335103 UTC,2022-03-07 15:57:23.335103+00:00,1.955790,1.021672,-1.656549,0.0,1.000000,0.000000,1.000000,0.000000,9.659258e-01,-2.588190e-01,9.659258e-01,-2.588190e-01,-0.420851,25923.007194,2176.550257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-05 10:18:56.756843 UTC,2022-08-05 10:18:56.756843+00:00,-0.326981,0.926779,-1.657293,0.0,-0.900969,-0.433884,-0.900969,-0.433884,-1.836970e-16,-1.000000e+00,-1.836970e-16,-1.000000e+00,1.834261,25923.007194,2176.550257
2022-08-05 19:32:33.609302 UTC,2022-08-05 19:32:33.609302+00:00,-0.327340,-0.335524,-1.657512,0.0,-0.222521,-0.974928,-0.222521,-0.974928,7.071068e-01,7.071068e-01,7.071068e-01,7.071068e-01,1.834261,25923.007194,2176.550257
2022-08-06 14:12:08.658448 UTC,2022-08-06 14:12:08.658448+00:00,-0.357456,0.825755,-1.657460,0.0,-0.222521,-0.974928,-0.222521,-0.974928,8.660254e-01,-5.000000e-01,8.660254e-01,-5.000000e-01,1.834261,25923.007194,2176.550257
